In [1]:
cd ..

/Users/deepakduggirala/Documents/autonomous-robotics/translating-images-into-maps


In [2]:
%matplotlib inline
import matplotlib as mpl
mpl.rcParams['figure.dpi']= 120

In [3]:
import matplotlib.pyplot as plt

In [4]:
import os
from pathlib import Path

from collections import namedtuple, Counter
import itertools

import numpy as np

from nuscenes.nuscenes import NuScenes
import pickle
import lmdb
from PIL import Image
import io
import torch
from torchvision.transforms.functional import to_tensor, to_pil_image

import pandas as pd

In [5]:
from src.data.dataloader import nuScenesMaps, read_split

In [6]:
data_root = Path.resolve(Path('/Users/deepakduggirala/Documents/autonomous-robotics/v1.0-trainval_meta/'))
# nusc = NuScenes(version='v1.0-trainval', dataroot=data_root, verbose=False)

In [7]:
tokens = read_split(
            os.path.join(data_root, "splits", "{}.txt".format('train_roddick'))
        )
val_tokens = read_split(
            os.path.join(data_root, "splits", "{}.txt".format('val_roddick'))
        )

In [8]:
def get_cam_id(sample_token):
    sample_record = nusc.get("sample", sample_token)
    cam_token = sample_record["data"]["CAM_FRONT"]
    cam_record = nusc.get("sample_data", cam_token)
    cam_path = nusc.get_sample_data_path(cam_token)
    return Path(cam_path).stem

In [32]:
# token_cam_id_map = {token:get_cam_id(token) for token in tokens}

In [36]:
# val_token_cam_id_map = {token:get_cam_id(token) for token in val_tokens}

In [40]:
# with open('train_roddick_token_cam_id_map.pickle', 'wb') as f:
#     pickle.dump(token_cam_id_map, f)

# with open('val_roddick_token_cam_id_map.pickle', 'wb') as f:
#     pickle.dump(val_token_cam_id_map, f)  

In [11]:
with open('train_roddick_token_cam_id_map.pickle', 'rb') as f:
    token_cam_id_map = pickle.load(f)

with open('val_roddick_token_cam_id_map.pickle', 'rb') as f:
    val_token_cam_id_map = pickle.load(f)  
    
len(token_cam_id_map), len(val_token_cam_id_map)

(28008, 5981)

In [12]:
gtmaps_db_path = os.path.join(
            data_root, "lmdb",
            "semantic_maps_new_200x200"
        )
gtmaps_db = lmdb.open(
            path=gtmaps_db_path,
            readonly=True,
            readahead=False,
            max_spare_txns=128,
            lock=False,
        )

In [13]:
classes = [
            "drivable_area",
            "ped_crossing",
            "walkway",
            "carpark_area",
            "road_segment",
            "lane",
            "bus",
            "bicycle",
            "car",
            "construction_vehicle",
            "motorcycle",
            "trailer",
            "truck",
            "pedestrian",
            "trafficcone",
            "barrier",
        ] + ['lidar_ray_mask_dense']
len(classes)

17

In [14]:
fov_mask = to_tensor(Image.open(
            os.path.join(data_root, "lmdb", "semantic_maps_new_200x200", "fov_mask.png")
        ))

In [15]:
def merge_map_classes(mapsdict):
    classes_to_merge = ["drivable_area", "road_segment", "lane"]
    merged_class = 'drivable_area'
    maps2merge = torch.stack([mapsdict[k] for k in classes_to_merge])  # [n, 1, 200, 200]
    maps2merge = maps2merge.sum(dim=0)
    maps2merge = (maps2merge > 0).float()
    mapsdict[merged_class] = maps2merge
    del mapsdict['road_segment'], mapsdict['lane']
    return mapsdict

In [17]:
i=0
cam_id = token_cam_id_map[tokens[i]]

In [16]:
def get_ground_truth_pixel_count(cam_id):
    
    gtmaps_key = [pickle.dumps("{}___{}".format(cam_id, cls), 3) for cls in classes]
    with gtmaps_db.begin() as txn:
        value = [txn.get(key=key) for key in gtmaps_key]
        gtmaps = [Image.open(io.BytesIO(im)) for im in value]
    
    # each map is of shape [1, 200, 200]
    mapsdict = {cls: to_tensor(map) for cls, map in zip(classes, gtmaps)}
    # mapsdict["fov_mask"] = to_tensor(fov_mask)
    mapsdict = merge_map_classes(mapsdict)

    # Create visbility mask from lidar and fov masks
    lidar_ray_mask = mapsdict['lidar_ray_mask_dense']
    # fov_mask = mapsdict['fov_mask']
    vis_mask = lidar_ray_mask * fov_mask
    mapsdict['vis_mask'] = vis_mask

    del mapsdict['lidar_ray_mask_dense']#, mapsdict['fov_mask']
    # apply vis mask to maps
    mapsdict = {
        cls: cls_map * mapsdict['vis_mask'] for cls, cls_map in mapsdict.items()
    }
    
    # del mapsdict['vis_mask']
    
    return {cls: cls_map.numpy().sum() for cls, cls_map in mapsdict.items()}

In [18]:
get_ground_truth_pixel_count(cam_id)

{'drivable_area': 13358.0,
 'ped_crossing': 2426.0,
 'walkway': 2149.0,
 'carpark_area': 0.0,
 'bus': 0.0,
 'bicycle': 0.0,
 'car': 472.0,
 'construction_vehicle': 56.0,
 'motorcycle': 0.0,
 'trailer': 0.0,
 'truck': 114.0,
 'pedestrian': 42.0,
 'trafficcone': 0.0,
 'barrier': 165.0,
 'vis_mask': 22781.0}

In [19]:
train_pixel_counts = {token: get_ground_truth_pixel_count(token_cam_id_map[token]) for token in tokens}

In [20]:
val_pixel_counts = {token: get_ground_truth_pixel_count(val_token_cam_id_map[token]) for token in val_tokens}

In [21]:
# with open('train_pixel_counts.pickle', 'wb') as f:
#     pickle.dump(train_pixel_counts, f)

# with open('val_pixel_counts.pickle', 'wb') as f:
#     pickle.dump(val_pixel_counts, f)  

# start here

In [22]:
with open('train_pixel_counts.pickle', 'rb') as f:
    train_pixel_counts = pickle.load(f)

with open('val_pixel_counts.pickle', 'rb') as f:
    val_pixel_counts = pickle.load(f)  

In [23]:
def create_df(pixel_counts):
    df = pd.DataFrame(pixel_counts).T
    x = df.values
    rel_count = x/(x[:,-1].reshape(-1,1))
    df_rel = pd.DataFrame(rel_count, columns=df.columns, index=df.index)
    return df_rel.drop(columns=['vis_mask'])

In [24]:
train_pixel_counts_df = create_df(train_pixel_counts)
val_pixel_counts_df = create_df(val_pixel_counts)

In [203]:
def plot_class_pixel_area_dstbn(df):
    fig, axs = plt.subplots(ncols=4, nrows=4, figsize=(8,8))

    for i in range(4):
        for j in range(4):
            k = 4*i + j
            if k < df.shape[1]:
                ax = axs[i][j]
                col = df.columns[k]
                df[col].hist(ax=ax)
                ax.set_title(col)

    # access each axes object via axs.flat
    for ax in axs.flat:
        # check if something was plotted
        if not bool(ax.has_data()):
            fig.delaxes(ax)  # delete if nothing is plotted in the axes obj
    fig.tight_layout()

In [25]:
train_class_freq = (train_pixel_counts_df > 0).mean()
train_pixel_area_ratio = train_pixel_counts_df.mean()

val_class_freq = (val_pixel_counts_df > 0).mean()
val_pixel_area_ratio = val_pixel_counts_df.mean()

In [26]:
train_stats = pd.concat([train_class_freq, train_pixel_area_ratio], axis=1)
train_stats.columns = ['cls freq', 'mean rel. area']

val_stats = pd.concat([val_class_freq, val_pixel_area_ratio], axis=1)
val_stats.columns = ['cls freq', 'mean rel. area']
# train_stats.sort_values(by='cls freq', ascending=False)

In [27]:
best_iou = pd.read_csv('./best_iou.csv', index_col=0).T
best_iou

,train-26,train-40,val-26,val-40
drivable_area,84.9,87.0,71.1,72.7
ped_crossing,63.6,69.2,34.2,34.9
walkway,52.5,57.3,32.8,33.9
carpark_area,61.9,68.4,30.7,30.0
bus,65.6,71.0,29.9,24.8
bicycle,34.3,42.6,14.0,8.6
car,46.6,51.8,38.2,37.9
construction_vehicle,48.5,55.7,0.2,0.0
motorcycle,34.9,45.4,7.5,5.7
trailer,50.2,58.6,11.7,9.8


In [32]:
train_stats['train-26-iou'] = best_iou['train-26']
train_stats['val-26-iou'] = best_iou['val-26']
# train_stats['k'] = (train_stats['cls freq'] * train_stats['mean rel. area'])/(train_stats['cls freq'] + train_stats['mean rel. area'])
# train_stats.sort_values(by=['k'])
# train_stats.to_csv('class_distribution_train_set.csv')
train_stats['cls freq'] = train_stats['cls freq'].round(2)
train_stats['mean rel. area'] = train_stats['mean rel. area'].round(4)
train_stats.to_csv('class_distribution_train_set.csv')
train_stats




,cls freq,mean rel. area,train-26-iou,val-26-iou
drivable_area,1.00,0.6962,84.9,71.1
ped_crossing,0.49,0.0372,63.6,34.2
walkway,0.97,0.1288,52.5,32.8
carpark_area,0.30,0.0220,61.9,30.7
bus,0.09,0.0022,65.6,29.9
bicycle,0.08,0.0004,34.3,14.0
car,0.76,0.0204,46.6,38.2
construction_vehicle,0.06,0.0009,48.5,0.2
motorcycle,0.07,0.0002,34.9,7.5
trailer,0.08,0.0020,50.2,11.7


In [256]:
val_stats['val-26'] = best_iou['val-26']
val_stats.sort_values(by=['cls freq'])

,cls freq,mean rel. area,val-26
motorcycle,0.050660,0.000128,7.5
construction_vehicle,0.073399,0.003313,0.2
bus,0.074235,0.001585,29.9
trafficcone,0.075405,0.000074,10.2
barrier,0.081926,0.001033,15.9
bicycle,0.085437,0.000524,14.0
trailer,0.086608,0.002259,11.7
pedestrian,0.315666,0.000550,10.4
carpark_area,0.328540,0.031206,30.7
truck,0.344591,0.006351,19.8
